# Houston, TX Simulations
8760 rows per simulation<br>
50 files (removed cell-in-cell)

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
import pprint
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

foot-candles --> Lux (multiply by 10.7642)

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

In [2]:
# 2018 Egrid Costs (elec $/kWh, gas $/therm), change values per city
gas = 1.11
elec = 0.112

---

## Combine CSV Files in Directory

In [3]:
# Create variable for files in directory
files = [f for f in os.listdir("data/hou/") if f.endswith(".csv")]

# files

In [4]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_data = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/hou/" + file)
    all_data = pd.concat([all_data, df])
    
#all_data

# 438000  rows, 73 columns

In [5]:
# all_data.dtypes

In [6]:
# Get wanted columns
all_data1 = all_data[["Scenario",
               "Date/Time",
              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
              "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)",
              "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)",
              "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) "]]

#all_data1

In [7]:
all_data1 = all_data1.dropna().reset_index(drop=True)

#all_data1

#438000  rows, 73 columns

-----

## Get the Wanted Data (columns)

In [8]:
# Rename Columns
all_data2 = all_data1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#all_data2

#438000  rows, 6 columns

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>


In [9]:
# Copy dataframe to modify and leave original df intact
get_annual = all_data2.copy(deep=True)

#get_annual

In [10]:
# Convert Joules to KBtu in dataframe
get_annual["FanEnergy(kBtu)"] = get_annual["FanEnergy[J](Hourly)"] * .00000094781712
get_annual["HeatingEnergy(kBtu)"] = get_annual["HeatingEnergy[J](Hourly)"] * .00000094781712
get_annual["CoolingEnergy(kBtu)"] = get_annual["CoolingEnergy[J](Hourly) "] * .00000094781712

#get_annual

In [11]:
# Drop columns
get_annual.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

get_annual

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu)
0,Clear/Clear,01/01 01:00:00,0.0,0.580415,17.656572,0.0
1,Clear/Clear,01/01 02:00:00,0.0,0.634104,19.289833,0.0
2,Clear/Clear,01/01 03:00:00,0.0,0.678973,20.654778,0.0
3,Clear/Clear,01/01 04:00:00,0.0,0.722009,21.963976,0.0
4,Clear/Clear,01/01 05:00:00,0.0,0.742622,22.591009,0.0
...,...,...,...,...,...,...
437995,lowE(2)/Clear-BBG75,12/31 20:00:00,0.0,0.000000,0.000000,0.0
437996,lowE(2)/Clear-BBG75,12/31 21:00:00,0.0,0.000000,0.000000,0.0
437997,lowE(2)/Clear-BBG75,12/31 22:00:00,0.0,0.011634,0.353927,0.0
437998,lowE(2)/Clear-BBG75,12/31 23:00:00,0.0,0.145747,4.433728,0.0


---

## Get Annual Cost Data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>
Add cost multiplier, variables defined above

In [12]:
# Get annual costs
get_annual_costs = all_data2.copy(deep=True)

In [13]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs["FanEnergy($)"] = (get_annual_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_annual_costs["HeatingEnergy($)"] = ((get_annual_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_annual_costs["CoolingEnergy($)"] = (get_annual_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_annual_costs

In [14]:
# Drop columns
get_annual_costs.drop(columns = ["Illuminance(lux)","FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

get_annual_costs

,Scenario,Date_Time,FanEnergy($),HeatingEnergy($),CoolingEnergy($)
0,Clear/Clear,01/01 01:00:00,0.019052,0.195988,0.0
1,Clear/Clear,01/01 02:00:00,0.020814,0.214117,0.0
2,Clear/Clear,01/01 03:00:00,0.022287,0.229268,0.0
3,Clear/Clear,01/01 04:00:00,0.023699,0.243800,0.0
4,Clear/Clear,01/01 05:00:00,0.024376,0.250760,0.0
...,...,...,...,...,...
437995,lowE(2)/Clear-BBG75,12/31 20:00:00,0.000000,0.000000,0.0
437996,lowE(2)/Clear-BBG75,12/31 21:00:00,0.000000,0.000000,0.0
437997,lowE(2)/Clear-BBG75,12/31 22:00:00,0.000382,0.003929,0.0
437998,lowE(2)/Clear-BBG75,12/31 23:00:00,0.004784,0.049214,0.0


---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Add baseline data <br>
Export combine file

In [15]:
# Combine data for energy and cost
allAnnualData = pd.merge(get_annual, get_annual_costs)

allAnnualData

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu),FanEnergy($),HeatingEnergy($),CoolingEnergy($)
0,Clear/Clear,01/01 01:00:00,0.0,0.580415,17.656572,0.0,0.019052,0.195988,0.0
1,Clear/Clear,01/01 02:00:00,0.0,0.634104,19.289833,0.0,0.020814,0.214117,0.0
2,Clear/Clear,01/01 03:00:00,0.0,0.678973,20.654778,0.0,0.022287,0.229268,0.0
3,Clear/Clear,01/01 04:00:00,0.0,0.722009,21.963976,0.0,0.023699,0.243800,0.0
4,Clear/Clear,01/01 05:00:00,0.0,0.742622,22.591009,0.0,0.024376,0.250760,0.0
...,...,...,...,...,...,...,...,...,...
437995,lowE(2)/Clear-BBG75,12/31 20:00:00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
437996,lowE(2)/Clear-BBG75,12/31 21:00:00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
437997,lowE(2)/Clear-BBG75,12/31 22:00:00,0.0,0.011634,0.353927,0.0,0.000382,0.003929,0.0
437998,lowE(2)/Clear-BBG75,12/31 23:00:00,0.0,0.145747,4.433728,0.0,0.004784,0.049214,0.0


In [16]:
# Add energy and cost totals
totalEnergy = allAnnualData["FanEnergy(kBtu)"] + allAnnualData["HeatingEnergy(kBtu)"] + allAnnualData["CoolingEnergy(kBtu)"]
allAnnualData["AnnualEnergy(kBtu)"] = totalEnergy

totalCost = allAnnualData["FanEnergy($)"] + allAnnualData["HeatingEnergy($)"] + allAnnualData["CoolingEnergy($)"]
allAnnualData["AnnualCost($)"] = totalCost

#allAnnualData


### Establish Annual Energy and Cost Baseline Data


In [17]:
# Establish energy target. % improvement from the average of Heat + Cool + Fan of Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees).
baseline_annual = allAnnualData[allAnnualData["Scenario"] == "Clear/Clear-VB0"]

#baseline_annual

In [18]:
annualEnergy_fan = baseline_annual["FanEnergy(kBtu)"].sum()
annualEnergy_fan

5535.829631227591

In [19]:
annualEnergy_cool = baseline_annual["CoolingEnergy(kBtu)"].sum()
annualEnergy_cool

31170.891245358252

In [20]:
annualEnergy_heat = baseline_annual["HeatingEnergy(kBtu)"].sum()
annualEnergy_heat

19289.426003497585

In [21]:
annualEnergy = (annualEnergy_fan + annualEnergy_cool + annualEnergy_heat)
annualEnergy

55996.14688008343

In [22]:
annualCost_fan = baseline_annual["FanEnergy($)"].sum()
annualCost_fan

181.70784980870937

In [23]:
annualCost_cool = baseline_annual["CoolingEnergy($)"].sum()
annualCost_cool

1023.1520841726409

In [24]:
annualCost_heat = baseline_annual["HeatingEnergy($)"].sum()
annualCost_heat

214.11262863882322

In [25]:
annualCost = (annualCost_fan + annualCost_cool + annualCost_heat)
annualCost

1418.9725626201734

---
## Group and Export Annual Data
Group dataframe by scenario<br>
Add baselines to dataframe<br>
Export to file

In [26]:
# Group by scenario
annualData = allAnnualData.groupby(["Scenario"], as_index=True)

#annualData.sum().round(2)

In [27]:
annualDataFinal = annualData.sum().round(2)

#annualDataFinal

In [28]:
# Add columns for delta from base
annualDataFinal["DeltaEnergyFromBase"] = annualDataFinal["AnnualEnergy(kBtu)"] - annualEnergy
annualDataFinal["DeltaCostFromBase"] = annualDataFinal["AnnualCost($)"] - annualCost

annualDataFinal.round(2)

,Illuminance(lux),FanEnergy(kBtu),HeatingEnergy(kBtu),CoolingEnergy(kBtu),FanEnergy($),HeatingEnergy($),CoolingEnergy($),AnnualEnergy(kBtu),AnnualCost($),DeltaEnergyFromBase,DeltaCostFromBase
Scenario,,,,,,,,,,,
Clear/Clear,19805736.58,6010.00,19906.00,33950.04,197.27,220.96,1114.37,59866.04,1532.60,3869.89,113.63
Clear/Clear-BBG-15,7493919.04,5377.44,22079.26,29665.12,176.51,245.08,973.73,57121.82,1395.32,1125.67,-23.65
Clear/Clear-BBG-30,13009026.79,5394.21,21412.56,29882.44,177.06,237.68,980.86,56689.20,1395.60,693.05,-23.37
Clear/Clear-BBG-45,15251212.03,5282.05,21149.57,29236.82,173.38,234.76,959.67,55668.44,1367.81,-327.71,-51.16
Clear/Clear-BBG-60,10164872.86,5009.88,21366.82,27523.95,164.44,237.17,903.45,53900.66,1305.06,-2095.49,-113.91
Clear/Clear-BBG-75,2384739.50,4615.65,22000.47,24961.45,151.50,244.21,819.33,51577.58,1215.04,-4418.57,-203.93
Clear/Clear-BBG0,4097349.84,5280.23,22746.31,28948.01,173.32,252.48,950.19,56974.55,1375.99,978.40,-42.98
Clear/Clear-BBG15,2856163.77,5153.71,23031.01,28115.28,169.17,255.64,922.85,56299.99,1347.66,303.84,-71.31
Clear/Clear-BBG30,1818722.97,4996.76,22982.69,27153.03,164.01,255.11,891.27,55132.48,1310.39,-863.67,-108.58


In [29]:
# Export grouped file to csv
annualDataFinal.to_csv("../Scraper_Output/hou_annual_data.csv", header=True, index=True)

-----

## Get Monthly Energy Data
Change units from Joules to therms and kWh<br>
Drop unwanted columns

In [30]:
# Copy dataframe to modify and leave original df intact
get_monthly = all_data2.copy(deep=True)

#get_monthly

In [31]:
# Convert Joules to kWh and therms in new dataframe
get_monthly["FanEnergy(kWh)"] = get_monthly["FanEnergy[J](Hourly)"] * .000000277777778
get_monthly["HeatingEnergy(therms)"] = get_monthly["HeatingEnergy[J](Hourly)"] * .0000000094781712
get_monthly["CoolingEnergy(kWh)"] = get_monthly["CoolingEnergy[J](Hourly) "] * .000000277777778

#get_monthly

In [32]:
# Drop columns
get_monthly.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

get_monthly

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh)
0,Clear/Clear,01/01 01:00:00,0.0,0.170103,0.176566,0.0
1,Clear/Clear,01/01 02:00:00,0.0,0.185838,0.192898,0.0
2,Clear/Clear,01/01 03:00:00,0.0,0.198987,0.206548,0.0
3,Clear/Clear,01/01 04:00:00,0.0,0.211600,0.219640,0.0
4,Clear/Clear,01/01 05:00:00,0.0,0.217641,0.225910,0.0
...,...,...,...,...,...,...
437995,lowE(2)/Clear-BBG75,12/31 20:00:00,0.0,0.000000,0.000000,0.0
437996,lowE(2)/Clear-BBG75,12/31 21:00:00,0.0,0.000000,0.000000,0.0
437997,lowE(2)/Clear-BBG75,12/31 22:00:00,0.0,0.003410,0.003539,0.0
437998,lowE(2)/Clear-BBG75,12/31 23:00:00,0.0,0.042714,0.044337,0.0


In [33]:
# get_monthly["Month"] = get_monthly["Date_Time"].apply(lambda x: str(x)[0:3])

# get_monthly

## Get Monthly Cost Data

Change units from Joules to therms and kWh<br>
Add cost multiplier, variable needs to be defined above

In [34]:
# Copy dataframe to modify and leave original df intact
get_monthly_costs = all_data2.copy(deep=True)

#get_monthly

In [35]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_monthly_costs["FanEnergy($)"] = (get_monthly_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_monthly_costs["HeatingEnergy($)"] = ((get_monthly_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_monthly_costs["CoolingEnergy($)"] = (get_monthly_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_monthly_costs

In [36]:
# Drop columns
get_monthly_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

get_monthly_costs

,Scenario,Date_Time,Illuminance(lux),FanEnergy($),HeatingEnergy($),CoolingEnergy($)
0,Clear/Clear,01/01 01:00:00,0.0,0.019052,0.195988,0.0
1,Clear/Clear,01/01 02:00:00,0.0,0.020814,0.214117,0.0
2,Clear/Clear,01/01 03:00:00,0.0,0.022287,0.229268,0.0
3,Clear/Clear,01/01 04:00:00,0.0,0.023699,0.243800,0.0
4,Clear/Clear,01/01 05:00:00,0.0,0.024376,0.250760,0.0
...,...,...,...,...,...,...
437995,lowE(2)/Clear-BBG75,12/31 20:00:00,0.0,0.000000,0.000000,0.0
437996,lowE(2)/Clear-BBG75,12/31 21:00:00,0.0,0.000000,0.000000,0.0
437997,lowE(2)/Clear-BBG75,12/31 22:00:00,0.0,0.000382,0.003929,0.0
437998,lowE(2)/Clear-BBG75,12/31 23:00:00,0.0,0.004784,0.049214,0.0


---

## Combine Monthly Energy and Cost Data
Combine two dfs<br>
Create Month Column<br>
Establish baseline <br>
Export combine file

In [37]:
# Combine data for energy and cost
allMonthlyData = pd.merge(get_monthly, get_monthly_costs, how="inner")

allMonthlyData

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh),FanEnergy($),HeatingEnergy($),CoolingEnergy($)
0,Clear/Clear,01/01 01:00:00,0.0,0.170103,0.176566,0.0,0.019052,0.195988,0.0
1,Clear/Clear,01/01 02:00:00,0.0,0.185838,0.192898,0.0,0.020814,0.214117,0.0
2,Clear/Clear,01/01 03:00:00,0.0,0.198987,0.206548,0.0,0.022287,0.229268,0.0
3,Clear/Clear,01/01 04:00:00,0.0,0.211600,0.219640,0.0,0.023699,0.243800,0.0
4,Clear/Clear,01/01 05:00:00,0.0,0.217641,0.225910,0.0,0.024376,0.250760,0.0
...,...,...,...,...,...,...,...,...,...
437995,lowE(2)/Clear-BBG75,12/31 20:00:00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
437996,lowE(2)/Clear-BBG75,12/31 21:00:00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
437997,lowE(2)/Clear-BBG75,12/31 22:00:00,0.0,0.003410,0.003539,0.0,0.000382,0.003929,0.0
437998,lowE(2)/Clear-BBG75,12/31 23:00:00,0.0,0.042714,0.044337,0.0,0.004784,0.049214,0.0


In [38]:
# Add cost totals, don't total energy...in different units
totalCostMonthly = allMonthlyData["FanEnergy($)"] + allMonthlyData["HeatingEnergy($)"] + allMonthlyData["CoolingEnergy($)"]
allMonthlyData["MonthlyCost($)"] = totalCostMonthly

#allMonthlyData

In [39]:
allMonthlyData["Month"] = allMonthlyData.Date_Time.str.slice(0,3)

allMonthlyData

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh),FanEnergy($),HeatingEnergy($),CoolingEnergy($),MonthlyCost($),Month
0,Clear/Clear,01/01 01:00:00,0.0,0.170103,0.176566,0.0,0.019052,0.195988,0.0,0.215039,01
1,Clear/Clear,01/01 02:00:00,0.0,0.185838,0.192898,0.0,0.020814,0.214117,0.0,0.234931,01
2,Clear/Clear,01/01 03:00:00,0.0,0.198987,0.206548,0.0,0.022287,0.229268,0.0,0.251555,01
3,Clear/Clear,01/01 04:00:00,0.0,0.211600,0.219640,0.0,0.023699,0.243800,0.0,0.267499,01
4,Clear/Clear,01/01 05:00:00,0.0,0.217641,0.225910,0.0,0.024376,0.250760,0.0,0.275136,01
...,...,...,...,...,...,...,...,...,...,...,...
437995,lowE(2)/Clear-BBG75,12/31 20:00:00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,12
437996,lowE(2)/Clear-BBG75,12/31 21:00:00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,12
437997,lowE(2)/Clear-BBG75,12/31 22:00:00,0.0,0.003410,0.003539,0.0,0.000382,0.003929,0.0,0.004310,12
437998,lowE(2)/Clear-BBG75,12/31 23:00:00,0.0,0.042714,0.044337,0.0,0.004784,0.049214,0.0,0.053998,12



### Establish Monthly Energy and Cost Baseline Data


In [40]:
# Establish energy target. % improvement from the average of Heat + Cool + Fan of Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees).
baseline_monthly = allMonthlyData[allMonthlyData["Scenario"] == "Clear/Clear-VB0"]

#baseline_monthly

In [41]:
monthlyEnergy_fan = baseline_monthly["FanEnergy(kWh)"].sum()
monthlyEnergy_fan

1622.3915161491905

In [42]:
monthlyEnergy_cool = baseline_monthly["CoolingEnergy(kWh)"].sum()
monthlyEnergy_cool

9135.286465827152

In [43]:
monthlyEnergy_heat = baseline_monthly["HeatingEnergy(therms)"].sum()
monthlyEnergy_heat

192.89426003497584

In [44]:
monthlyCost_fan = baseline_monthly["FanEnergy($)"].sum()
monthlyCost_fan

181.70784980870937

In [45]:
monthlyCost_cool = baseline_monthly["CoolingEnergy($)"].sum()
monthlyCost_cool

1023.1520841726409

In [46]:
monthlyCost_heat = baseline_monthly["HeatingEnergy($)"].sum()
monthlyCost_heat

214.11262863882322

In [47]:
monthlyCost = (monthlyCost_fan + monthlyCost_cool + monthlyCost_heat)
monthlyCost

1418.9725626201734

## Group and Export Monthly Data
Group dataframe by scenario and month<br>
Add baselines to dataframe<br>
Export to file

In [48]:
# Group by scenario and date/time for export to graphing
MonthlyData2 = allMonthlyData.groupby(["Scenario", "Month"], as_index=True)

#MonthlyData.sum()

In [49]:
MonthlyDataFinal = MonthlyData2.sum()

#MonthlyDataFinal

In [50]:
# Add columns for delta from base
MonthlyDataFinal["DeltaFanFromBase"] = MonthlyDataFinal["FanEnergy(kWh)"] - monthlyEnergy_fan
MonthlyDataFinal["DeltaCoolFromBase"] = MonthlyDataFinal["CoolingEnergy(kWh)"] - monthlyEnergy_cool
MonthlyDataFinal["DeltaHeatFromBase"] = MonthlyDataFinal["HeatingEnergy(therms)"] - monthlyEnergy_heat
MonthlyDataFinal["DeltaCostFromBase"] = MonthlyDataFinal["MonthlyCost($)"] - monthlyCost

MonthlyDataFinal.round(2)

Illuminance(lux)  FanEnergy(kWh)  \
Scenario           Month                                     
Clear/Clear         01          2777225.12           69.94   
                    02          2892140.59           67.69   
                    03           799290.31           73.45   
                    04           786791.63           99.46   
                    05           695624.67          163.67   
...                                    ...             ...   
lowE(2)/Clear-VB75  08            18951.34          201.91   
                    09            23994.76          161.22   
                    10            27297.51           63.38   
                    11            23439.65           41.44   
                    12            21040.17           52.08   

                          HeatingEnergy(therms)  CoolingEnergy(kWh)  \
Scenario           Month                                              
Clear/Clear         01                    59.40               71.57   
                    02                    43.59              148.87   
                    03                    22.43              303.16   
                    04                     4.09              581.29   
                    05                     0.11             1022.63   
...                                         ...                 ...   
lowE(2)/Clear-VB75  08                     0.00             1327.87   
                    09                     0.00             1039.68   
                    10                     5.29              362.10   
                    11                    17.96              147.06   
                    12                    48.34               33.16   

                          FanEnergy($)  HeatingEnergy($)  CoolingEnergy($)  \
Scenario           Month                                                     
Clear/Clear         01            7.83             65.94              8.02   
                    02            7.58             48.38             16.67   
                    03            8.23             24.90             33.95   
                    04           11.14              4.54             65.10   
                    05           18.33              0.13            114.53   
...                                ...               ...               ...   
lowE(2)/Clear-VB75  08           22.61              0.00            148.72   
                    09           18.06              0.00            116.44   
                    10            7.10              5.87             40.56   
                    11            4.64             19.93             16.47   
                    12            5.83             53.65              3.71   

                          MonthlyCost($)  DeltaFanFromBase  DeltaCoolFromBase  \
Scenario           Month                                                        
Clear/Clear         01             81.79          -1552.45           -9063.72   
                    02             72.63          -1554.71           -8986.42   
                    03             67.08          -1548.94           -8832.12   
                    04             80.78          -1522.93           -8554.00   
                    05            132.99          -1458.72           -8112.66   
...                                  ...               ...                ...   
lowE(2)/Clear-VB75  08            171.34          -1420.48           -7807.42   
                    09            134.50          -1461.17           -8095.60   
                    10             53.52          -1559.02           -8773.19   
                    11             41.04          -1580.95           -8988.23   
                    12             63.20          -1570.31           -9102.12   

                          DeltaHeatFromBase  DeltaCostFromBase  
Scenario           Month                                        
Clear/Clear         01              -133.49           -1337.18  
                    02         

In [51]:
# Drop Illuminance column...don't need the totals
MonthlyDataFinal.drop(columns = ["Illuminance(lux)"], axis=1, inplace=True)

In [52]:
# Export grouped file to csv (used before grouped by month)
MonthlyDataFinal.round(2).to_csv("../Scraper_Output/hou_monthly_data.csv", header=True, index=True)

## Get Monthly Average Illuminance
Get just the illuminance column<br>
Group by scenario <br>
Groups by month <br>

In [53]:
Illuminance = allMonthlyData[["Scenario", "Date_Time", "Illuminance(lux)", "Month"]]

#Illuminance

In [54]:
# Group by scenario
MonthlyIlluminance = Illuminance.groupby(["Scenario", "Month"])

MonthlyIlluminance.mean()

Illuminance(lux)
Scenario           Month                  
Clear/Clear         01         3732.829463
                    02         4303.780642
                    03         1074.314935
                    04         1092.766152
                    05          934.979390
...                                    ...
lowE(2)/Clear-VB75  08           25.472227
                    09           33.326059
                    10           36.690200
                    11           32.555067
                    12           28.279800

[600 rows x 1 columns]

In [55]:
# Export grouped file to csv
MonthlyIlluminance.mean().round(2).to_csv("../Scraper_Output/hou_monthly_illuminance.csv", header=True, index=True)

------

## Get Hourly Energy Data
Change units from Joules to therms and kWh<br>
Drop unwanted columns

In [56]:
# Copy dataframe to modify and leave original df intact
all_data3 = all_data1.copy(deep=True)

#all_data3

In [57]:
# Get wanted columns
# Include hourly illumance
all_data3 = all_data3[["Scenario",
               "Date/Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) "
                      ]]

#all_data3

In [58]:
# Rename Columns
get_hourly = all_data3.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#get_hourly

In [59]:
# Convert Joules to kWh and therms in new dataframe
get_hourly["FanEnergy(kWh)"] = get_hourly["FanEnergy[J](Hourly)"] * .000000277777778
get_hourly["HeatingEnergy(therms)"] = get_hourly["HeatingEnergy[J](Hourly)"] * .0000000094781712
get_hourly["CoolingEnergy(kWh)"] = get_hourly["CoolingEnergy[J](Hourly) "] * .000000277777778

#get_hourly

In [60]:
# Drop columns
get_hourly.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly

------

## Get Hourly Cost Data
Change units from Joules to therms and kWh<br>
Add cost multiplier, variable needs to be defined above

In [61]:
# Copy dataframe to modify and leave original df intact
get_hourly_costs = all_data3.copy(deep=True)

#get_hourly_costs

In [62]:
# Rename Columns
get_hourly_costs = get_hourly_costs.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

# get_hourly_costs

In [63]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_hourly_costs["FanEnergy($)"] = (get_hourly_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_hourly_costs["HeatingEnergy($)"] = ((get_hourly_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_hourly_costs["CoolingEnergy($)"] = (get_hourly_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_monthly_costs

In [64]:
# Drop columns
get_hourly_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_hourly_costs

## Combine Hourly Energy and Cost Data
Combine two dfs<br>
Establish baseline <br>
Export combine file

In [65]:
# Combine data for energy and cost
allHourlyData = pd.merge(get_hourly, get_hourly_costs, how="inner")

#allHourlyData

In [66]:
# Add energy cost totals

totalCosthourly = allHourlyData["FanEnergy($)"] + allHourlyData["HeatingEnergy($)"] + allHourlyData["CoolingEnergy($)"]
allHourlyData["HourlyCost($)"] = totalCosthourly

allHourlyData

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh),FanEnergy($),HeatingEnergy($),CoolingEnergy($),HourlyCost($)
0,Clear/Clear,01/01 01:00:00,0.0,0.170103,0.176566,0.0,0.019052,0.195988,0.0,0.215039
1,Clear/Clear,01/01 02:00:00,0.0,0.185838,0.192898,0.0,0.020814,0.214117,0.0,0.234931
2,Clear/Clear,01/01 03:00:00,0.0,0.198987,0.206548,0.0,0.022287,0.229268,0.0,0.251555
3,Clear/Clear,01/01 04:00:00,0.0,0.211600,0.219640,0.0,0.023699,0.243800,0.0,0.267499
4,Clear/Clear,01/01 05:00:00,0.0,0.217641,0.225910,0.0,0.024376,0.250760,0.0,0.275136
...,...,...,...,...,...,...,...,...,...,...
437995,lowE(2)/Clear-BBG75,12/31 20:00:00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
437996,lowE(2)/Clear-BBG75,12/31 21:00:00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
437997,lowE(2)/Clear-BBG75,12/31 22:00:00,0.0,0.003410,0.003539,0.0,0.000382,0.003929,0.0,0.004310
437998,lowE(2)/Clear-BBG75,12/31 23:00:00,0.0,0.042714,0.044337,0.0,0.004784,0.049214,0.0,0.053998



### Establish Hourly Energy and Cost Baseline Data


In [67]:
# Establish energy target. % improvement from the average of Heat + Cool + Fan of Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees).
baseline_hourly = allHourlyData[allHourlyData["Scenario"] == "Clear/Clear-VB0"]

#baseline_hourly

In [68]:
hourlyEnergy_fan = baseline_hourly["FanEnergy(kWh)"].mean()
hourlyEnergy_fan

0.18520451097593474

In [69]:
hourlyEnergy_cool = baseline_hourly["CoolingEnergy(kWh)"].mean()
hourlyEnergy_cool

1.0428409207565226

In [70]:
hourlyEnergy_heat = baseline_hourly["HeatingEnergy(therms)"].mean()
hourlyEnergy_heat

0.022019892698056626

In [71]:
hourlyCost_fan = baseline_hourly["FanEnergy($)"].mean()
hourlyCost_fan

0.02074290522930468

In [72]:
hourlyCost_cool = baseline_hourly["CoolingEnergy($)"].mean()
hourlyCost_cool

0.11679818312473092

In [73]:
hourlyCost_heat = baseline_hourly["HeatingEnergy($)"].mean()
hourlyCost_heat

0.024442080894842858

In [74]:
hourlyCost = (hourlyCost_fan + hourlyCost_cool + hourlyCost_heat)
hourlyCost

0.16198316924887846

## Export Hourly Data

Add baselines to dataframe<br>
Export to file

In [75]:
# Add columns for delta from base
allHourlyData["DeltaFanFromBase"] = allHourlyData["FanEnergy(kWh)"] - hourlyEnergy_fan
allHourlyData["DeltaCoolFromBase"] = allHourlyData["CoolingEnergy(kWh)"] - hourlyEnergy_cool
allHourlyData["DeltaHeatFromBase"] = allHourlyData["HeatingEnergy(therms)"] - hourlyEnergy_heat
allHourlyData["DeltaCostFromBase"] = allHourlyData["HourlyCost($)"] - hourlyCost

allHourlyData.round(2)

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh),FanEnergy($),HeatingEnergy($),CoolingEnergy($),HourlyCost($),DeltaFanFromBase,DeltaCoolFromBase,DeltaHeatFromBase,DeltaCostFromBase
0,Clear/Clear,01/01 01:00:00,0.0,0.17,0.18,0.0,0.02,0.20,0.0,0.22,-0.02,-1.04,0.15,0.05
1,Clear/Clear,01/01 02:00:00,0.0,0.19,0.19,0.0,0.02,0.21,0.0,0.23,0.00,-1.04,0.17,0.07
2,Clear/Clear,01/01 03:00:00,0.0,0.20,0.21,0.0,0.02,0.23,0.0,0.25,0.01,-1.04,0.18,0.09
3,Clear/Clear,01/01 04:00:00,0.0,0.21,0.22,0.0,0.02,0.24,0.0,0.27,0.03,-1.04,0.20,0.11
4,Clear/Clear,01/01 05:00:00,0.0,0.22,0.23,0.0,0.02,0.25,0.0,0.28,0.03,-1.04,0.20,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437995,lowE(2)/Clear-BBG75,12/31 20:00:00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,-0.19,-1.04,-0.02,-0.16
437996,lowE(2)/Clear-BBG75,12/31 21:00:00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,-0.19,-1.04,-0.02,-0.16
437997,lowE(2)/Clear-BBG75,12/31 22:00:00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,-0.18,-1.04,-0.02,-0.16
437998,lowE(2)/Clear-BBG75,12/31 23:00:00,0.0,0.04,0.04,0.0,0.00,0.05,0.0,0.05,-0.14,-1.04,0.02,-0.11


In [76]:
# Export grouped file to csv for daily cost sums
allHourlyData.to_csv("../Scraper_Output/hou_daily.csv", header=True, index=False)